In [1]:
import datashader as ds
import datashader.transfer_functions as tf
import numpy as np
import pandas as pd
import dask.dataframe as dd

In [17]:
newDF = pd.DataFrame(dict([('a',[1]),('b',[2])]))
newDF.head()
newDF.append(dict([('a',3),('b',4)]),ignore_index=True)

,a,b
0,1,2
1,3,4


## In this notebook, we will be using the U.S. census .sf1 file for Washington, D.C. and the TIGER .shp file for D.C. blocks to visualize income category distributions through the district.

## Actually, screw that. Household income apparently doesn't even go down to the block level.

However, it does go down to the block-group level. You can find that data (for the District of Columbia) here: http://factfinder.census.gov/faces/tableservices/jsf/pages/productview.xhtml?pid=ACS_14_5YR_B19001&prodType=table

In [3]:
df = pd.read_csv("ACS_14_5YR_B19001_with_ann.csv")

In [4]:
df = df.drop(0)

In [5]:
df.head()

,GEO.id,GEO.id2,GEO.display-label,HD01_VD01,HD02_VD01,HD01_VD02,HD02_VD02,HD01_VD03,HD02_VD03,HD01_VD04,...,HD01_VD13,HD02_VD13,HD01_VD14,HD02_VD14,HD01_VD15,HD02_VD15,HD01_VD16,HD02_VD16,HD01_VD17,HD02_VD17
1,1500000US110010001001,110010001001,"Block Group 1, Census Tract 1, District of Col...",641,104,25,40,20,31,0,...,37,42,77,55,43,34,42,38,277,91
2,1500000US110010001002,110010001002,"Block Group 2, Census Tract 1, District of Col...",822,138,11,18,0,12,17,...,59,47,159,104,52,46,123,64,221,100
3,1500000US110010001003,110010001003,"Block Group 3, Census Tract 1, District of Col...",508,110,13,21,37,60,0,...,11,20,15,23,0,12,115,94,284,78
4,1500000US110010001004,110010001004,"Block Group 4, Census Tract 1, District of Col...",475,97,0,12,0,12,0,...,15,22,6,11,69,56,44,51,296,80
5,1500000US110010002011,110010002011,"Block Group 1, Census Tract 2.01, District of ...",4,5,0,12,0,12,0,...,0,12,0,12,0,12,0,12,4,5


## Lets find out what kind of population we're working with here

In [6]:
df.HD01_VD01.astype(int).sum()

267415

So we have the income information. But how the hell do we put that on a map? Luckily, TIGER shapefiles exist! We can find the one we need at http://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2015&layergroup=Block+Groups

In [7]:
District = ((-8584933.081216412, 4691868.155985658),(-8561511.126395036, 4721077.567819803))

x_range,y_range = District

plot_width  = int(1000)
plot_height = int(plot_width*7.0/12)

background = "black"

In [8]:
from functools import partial
from datashader.utils import export_image
from datashader.colors import colormap_select, Greys9, Hot, viridis, inferno
from IPython.core.display import HTML, display

export = partial(export_image, background = background)
cm = partial(colormap_select, reverse=(background!="black"))

display(HTML("<style>.container { width:100% !important; }</style>"))

We'll use this 'WebMerc' projection to convert all geodetic coordinates to Web Mercator for use with Data Shader

In [9]:
import pyproj
WebMerc=pyproj.Proj("+init=EPSG:3857")

# Now comes a very hard part. We need to take the census block data from our pandas dataframe and generate a random distribution of points equal to the population in each income category, and the points need to have coordinates geospatially within each census block

# NOTE: In census Area (land) column, data is square meters

# WAY AHEAD: Download .shp file from TIGER, use shapely and GDAL like in 
https://github.com/unorthodox123/RacialDotMap/blob/master/dotfile.py 

In [10]:
import sys
from osgeo import ogr
from shapely.wkb import loads
from shapely.geometry import *
from random import uniform

from globalmaptiles import GlobalMercator

# Important: When you get the .zip of your shapefiles from TIGER, extract all files into this projects working directory. You need all of them to correctly load with ogr

Download DC 2014 Block-Group shape file at https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2014&layergroup=Block+Groups

In [11]:
# sys.path.append("filepath")

# Create a GlobalMercator object for later conversions    
merc = GlobalMercator()

## Shapefile available?
# driverName = "ESRI Shapefile"
# drv = ogr.GetDriverByName( driverName )
# if drv is None:
#     print("{0} driver not available.\n".format(driverName))
# else:
#     print("{0} driver IS available.\n".format(driverName))

ds = ogr.Open(r"/home/terminal/projects/tl_2014_11_bg.shp", 0)

In [12]:
 # Obtain the first (and only) layer in the shapefile
lyr = ds.GetLayerByIndex(0)

lyr.ResetReading()

# Obtain the field definitions in the shapefile layer

feat_defn = lyr.GetLayerDefn()
field_defns = [feat_defn.GetFieldDefn(i) for i in range(feat_defn.GetFieldCount())]

In [13]:
# Obtain the index of the field for the count for whites, blacks, Asians, 
# Others, and Hispanics.

for i, defn in enumerate(field_defns):
    if defn.GetName() == 'GEOID':
        geo_field = i

In [23]:

out_csv = pd.DataFrame(columns=('GEO.id2', 'x', 'y', 'quadkey', 'income_category'))
out_csv = out_csv.append(dict([('GEO.id2',12),('x',13),('y',14),('quadkey',15),('income_category',16)]),ignore_index=True)
out_csv.append(dict([('GEO.id2',13),('x',13),('y',14),('quadkey',15),('income_category',16)]),ignore_index=True)

,GEO.id2,x,y,quadkey,income_category
0,12.0,13.0,14.0,15.0,16.0
1,13.0,13.0,14.0,15.0,16.0


### This next block of code will create the data that we need in order to visualize income-category distribution within D.C. block-groups, however it is not optimized and will take a very long time to run. Do not run yet.

In [ ]:
n_features = len(lyr)
out_csv = pd.DataFrame(columns=('GEO.id2', 'x', 'y', 'quadkey', 'income_category'))
for j, feat in enumerate( lyr ):

    # Print a progress read-out for every 1000 features and export to hard disk

    if j % 10 == 0:
        #print(feat.GetField(geo_field))
        print("{0}/{1}".format(j+1,n_features))
        print(out_csv.count())

    # Obtain total population and income bracket count for each block group
    geo = feat.GetField(geo_field)
    total_pop = int(df[df['GEO.id2'] == geo].HD01_VD01)
    lt_10k = int(df[df['GEO.id2'] == geo].HD01_VD02)
    bt_10k_15k = int(df[df['GEO.id2'] == geo].HD01_VD03)
    bt_15k_20k = int(df[df['GEO.id2'] == geo].HD01_VD04)
    bt_20k_25k = int(df[df['GEO.id2'] == geo].HD01_VD05)
    bt_25k_30k = int(df[df['GEO.id2'] == geo].HD01_VD06)
    bt_30k_35k = int(df[df['GEO.id2'] == geo].HD01_VD07)
    bt_35k_40k = int(df[df['GEO.id2'] == geo].HD01_VD08)
    bt_40k_45k = int(df[df['GEO.id2'] == geo].HD01_VD09)
    bt_45k_50k = int(df[df['GEO.id2'] == geo].HD01_VD10)
    bt_50k_60k = int(df[df['GEO.id2'] == geo].HD01_VD11)
    bt_60k_75k = int(df[df['GEO.id2'] == geo].HD01_VD12)
    bt_75k_100k = int(df[df['GEO.id2'] == geo].HD01_VD13)
    bt_100k_125k = int(df[df['GEO.id2'] == geo].HD01_VD14)
    bt_125k_150k = int(df[df['GEO.id2'] == geo].HD01_VD15)
    bt_150k_200k = int(df[df['GEO.id2'] == geo].HD01_VD16)
    gt_200k = int(df[df['GEO.id2'] == geo].HD01_VD17)

    # Obtain the OGR polygon object from the feature

    geom = feat.GetGeometryRef()

    if geom is None:
        print("continued")
        continue

    # Convert the OGR Polygon into a Shapely Polygon

    poly = loads(geom.ExportToWkb())

    if poly is None:
        continue
    bbox = poly.bounds

    if not bbox:
        continue

    leftmost,bottommost,rightmost,topmost = bbox
    
    # Generate a point object within the census block for every person by income category   
    for i in range(lt_10k):
                
        # Choose a random longitude and latitude within the boundary box
        # and within the orginial ploygon of the census block

        while True:

            samplepoint = Point(uniform(leftmost, rightmost),uniform(bottommost, topmost))

            if samplepoint is None:
                break

            if poly.contains(samplepoint):
                break

        # Convert the longitude and latitude coordinates to meters and
        # a tile reference

        x, y = merc.LatLonToMeters(samplepoint.y,samplepoint.x)
        tx,ty = merc.MetersToTile(x, y, 21)

        # Create a unique quadkey for each point object
                
        quadkey = merc.QuadTree(tx, ty, 21)

        # Create categorical variable for the income category

        income_category = 'HD01_VD02'         

        # Export data to the csv file

#         out_csv.loc[pandas_idx] = [geo, x, y, quadkey,income_category]
        out_csv = out_csv.append(dict([('GEO.id2',geo),('x',x),('y',y),('quadkey',quadkey),('income_category',income_category)]),ignore_index=True)
#         pandas_idx += 1
        
    for i in range(bt_10k_15k):
                
        # Choose a random longitude and latitude within the boundary box
        # and within the orginial ploygon of the census block

        while True:

            samplepoint = Point(uniform(leftmost, rightmost),uniform(bottommost, topmost))

            if samplepoint is None:
                break

            if poly.contains(samplepoint):
                break

        # Convert the longitude and latitude coordinates to meters and
        # a tile reference

        x, y = merc.LatLonToMeters(samplepoint.y,samplepoint.x)
        tx,ty = merc.MetersToTile(x, y, 21)

        # Create a unique quadkey for each point object
                
        quadkey = merc.QuadTree(tx, ty, 21)

        # Create categorical variable for the income category

        income_category = 'HD01_VD03'         

        # Export data to the csv file

#         out_csv.loc[pandas_idx] = [geo, x, y, quadkey,income_category]
        out_csv = out_csv.append(dict([('GEO.id2',geo),('x',x),('y',y),('quadkey',quadkey),('income_category',income_category)]),ignore_index=True)
#         pandas_idx += 1
        
    for i in range(bt_15k_20k):
                
        # Choose a random longitude and latitude within the boundary box
        # and within the orginial ploygon of the census block

        while True:

            samplepoint = Point(uniform(leftmost, rightmost),uniform(bottommost, topmost))

            if samplepoint is None:
                break

            if poly.contains(samplepoint):
                break

        # Convert the longitude and latitude coordinates to meters and
        # a tile reference

        x, y = merc.LatLonToMeters(samplepoint.y,samplepoint.x)
        tx,ty = merc.MetersToTile(x, y, 21)

        # Create a unique quadkey for each point object
                
        quadkey = merc.QuadTree(tx, ty, 21)

        # Create categorical variable for the income category

        income_category = 'HD01_VD04'         

        # Export data to the csv file

#         out_csv.loc[pandas_idx] = [geo, x, y, quadkey,income_category]
        out_csv = out_csv.append(dict([('GEO.id2',geo),('x',x),('y',y),('quadkey',quadkey),('income_category',income_category)]),ignore_index=True)
#         pandas_idx += 1
        
    for i in range(bt_20k_25k):
                
        # Choose a random longitude and latitude within the boundary box
        # and within the orginial ploygon of the census block

        while True:

            samplepoint = Point(uniform(leftmost, rightmost),uniform(bottommost, topmost))

            if samplepoint is None:
                break

            if poly.contains(samplepoint):
                break

        # Convert the longitude and latitude coordinates to meters and
        # a tile reference

        x, y = merc.LatLonToMeters(samplepoint.y,samplepoint.x)
        tx,ty = merc.MetersToTile(x, y, 21)

        # Create a unique quadkey for each point object
                
        quadkey = merc.QuadTree(tx, ty, 21)

        # Create categorical variable for the income category

        income_category = 'HD01_VD05'         

        # Export data to the csv file

#         out_csv.loc[pandas_idx] = [geo, x, y, quadkey,income_category]
        out_csv = out_csv.append(dict([('GEO.id2',geo),('x',x),('y',y),('quadkey',quadkey),('income_category',income_category)]),ignore_index=True)
#         pandas_idx += 1
        
    for i in range(bt_25k_30k):
                
        # Choose a random longitude and latitude within the boundary box
        # and within the orginial ploygon of the census block

        while True:

            samplepoint = Point(uniform(leftmost, rightmost),uniform(bottommost, topmost))

            if samplepoint is None:
                break

            if poly.contains(samplepoint):
                break

        # Convert the longitude and latitude coordinates to meters and
        # a tile reference

        x, y = merc.LatLonToMeters(samplepoint.y,samplepoint.x)
        tx,ty = merc.MetersToTile(x, y, 21)

        # Create a unique quadkey for each point object
                
        quadkey = merc.QuadTree(tx, ty, 21)

        # Create categorical variable for the income category

        income_category = 'HD01_VD06'         

        # Export data to the csv file

#         out_csv.loc[pandas_idx] = [geo, x, y, quadkey,income_category]
        out_csv = out_csv.append(dict([('GEO.id2',geo),('x',x),('y',y),('quadkey',quadkey),('income_category',income_category)]),ignore_index=True)
#         pandas_idx += 1
        
    for i in range(bt_30k_35k):
                
        # Choose a random longitude and latitude within the boundary box
        # and within the orginial ploygon of the census block

        while True:

            samplepoint = Point(uniform(leftmost, rightmost),uniform(bottommost, topmost))

            if samplepoint is None:
                break

            if poly.contains(samplepoint):
                break

        # Convert the longitude and latitude coordinates to meters and
        # a tile reference

        x, y = merc.LatLonToMeters(samplepoint.y,samplepoint.x)
        tx,ty = merc.MetersToTile(x, y, 21)

        # Create a unique quadkey for each point object
                
        quadkey = merc.QuadTree(tx, ty, 21)

        # Create categorical variable for the income category

        income_category = 'HD01_VD07'         

        # Export data to the csv file

#         out_csv.loc[pandas_idx] = [geo, x, y, quadkey,income_category]
        out_csv = out_csv.append(dict([('GEO.id2',geo),('x',x),('y',y),('quadkey',quadkey),('income_category',income_category)]),ignore_index=True)
#         pandas_idx += 1
        
    for i in range(bt_35k_40k):
                
        # Choose a random longitude and latitude within the boundary box
        # and within the orginial ploygon of the census block

        while True:

            samplepoint = Point(uniform(leftmost, rightmost),uniform(bottommost, topmost))

            if samplepoint is None:
                break

            if poly.contains(samplepoint):
                break

        # Convert the longitude and latitude coordinates to meters and
        # a tile reference

        x, y = merc.LatLonToMeters(samplepoint.y,samplepoint.x)
        tx,ty = merc.MetersToTile(x, y, 21)

        # Create a unique quadkey for each point object
                
        quadkey = merc.QuadTree(tx, ty, 21)

        # Create categorical variable for the income category

        income_category = 'HD01_VD08'         

        # Export data to the csv file

#         out_csv.loc[pandas_idx] = [geo, x, y, quadkey,income_category]
        out_csv = out_csv.append(dict([('GEO.id2',geo),('x',x),('y',y),('quadkey',quadkey),('income_category',income_category)]),ignore_index=True)
#         pandas_idx += 1
        
    for i in range(bt_40k_45k):
                
        # Choose a random longitude and latitude within the boundary box
        # and within the orginial ploygon of the census block

        while True:

            samplepoint = Point(uniform(leftmost, rightmost),uniform(bottommost, topmost))

            if samplepoint is None:
                break

            if poly.contains(samplepoint):
                break

        # Convert the longitude and latitude coordinates to meters and
        # a tile reference

        x, y = merc.LatLonToMeters(samplepoint.y,samplepoint.x)
        tx,ty = merc.MetersToTile(x, y, 21)

        # Create a unique quadkey for each point object
                
        quadkey = merc.QuadTree(tx, ty, 21)

        # Create categorical variable for the income category

        income_category = 'HD01_VD09'         

        # Export data to the csv file

#         out_csv.loc[pandas_idx] = [geo, x, y, quadkey,income_category]
        out_csv = out_csv.append(dict([('GEO.id2',geo),('x',x),('y',y),('quadkey',quadkey),('income_category',income_category)]),ignore_index=True)
#         pandas_idx += 1
        
    for i in range(bt_45k_50k):
                
        # Choose a random longitude and latitude within the boundary box
        # and within the orginial ploygon of the census block

        while True:

            samplepoint = Point(uniform(leftmost, rightmost),uniform(bottommost, topmost))

            if samplepoint is None:
                break

            if poly.contains(samplepoint):
                break

        # Convert the longitude and latitude coordinates to meters and
        # a tile reference

        x, y = merc.LatLonToMeters(samplepoint.y,samplepoint.x)
        tx,ty = merc.MetersToTile(x, y, 21)

        # Create a unique quadkey for each point object
                
        quadkey = merc.QuadTree(tx, ty, 21)

        # Create categorical variable for the income category

        income_category = 'HD01_VD10'         

        # Export data to the csv file

#         out_csv.loc[pandas_idx] = [geo, x, y, quadkey,income_category]
        out_csv = out_csv.append(dict([('GEO.id2',geo),('x',x),('y',y),('quadkey',quadkey),('income_category',income_category)]),ignore_index=True)
#         pandas_idx += 1
        
    for i in range(bt_50k_60k):
                
        # Choose a random longitude and latitude within the boundary box
        # and within the orginial ploygon of the census block

        while True:

            samplepoint = Point(uniform(leftmost, rightmost),uniform(bottommost, topmost))

            if samplepoint is None:
                break

            if poly.contains(samplepoint):
                break

        # Convert the longitude and latitude coordinates to meters and
        # a tile reference

        x, y = merc.LatLonToMeters(samplepoint.y,samplepoint.x)
        tx,ty = merc.MetersToTile(x, y, 21)

        # Create a unique quadkey for each point object
                
        quadkey = merc.QuadTree(tx, ty, 21)

        # Create categorical variable for the income category

        income_category = 'HD01_VD11'         

        # Export data to the csv file

#         out_csv.loc[pandas_idx] = [geo, x, y, quadkey,income_category]
        out_csv = out_csv.append(dict([('GEO.id2',geo),('x',x),('y',y),('quadkey',quadkey),('income_category',income_category)]),ignore_index=True)
#         pandas_idx += 1
        
    for i in range(bt_60k_75k):
                
        # Choose a random longitude and latitude within the boundary box
        # and within the orginial ploygon of the census block

        while True:

            samplepoint = Point(uniform(leftmost, rightmost),uniform(bottommost, topmost))

            if samplepoint is None:
                break

            if poly.contains(samplepoint):
                break

        # Convert the longitude and latitude coordinates to meters and
        # a tile reference

        x, y = merc.LatLonToMeters(samplepoint.y,samplepoint.x)
        tx,ty = merc.MetersToTile(x, y, 21)

        # Create a unique quadkey for each point object
                
        quadkey = merc.QuadTree(tx, ty, 21)

        # Create categorical variable for the income category

        income_category = 'HD01_VD12'         

        # Export data to the csv file

#         out_csv.loc[pandas_idx] = [geo, x, y, quadkey,income_category]
        out_csv = out_csv.append(dict([('GEO.id2',geo),('x',x),('y',y),('quadkey',quadkey),('income_category',income_category)]),ignore_index=True)
#         pandas_idx += 1
        
    for i in range(bt_75k_100k):
                
        # Choose a random longitude and latitude within the boundary box
        # and within the orginial ploygon of the census block

        while True:

            samplepoint = Point(uniform(leftmost, rightmost),uniform(bottommost, topmost))

            if samplepoint is None:
                break

            if poly.contains(samplepoint):
                break

        # Convert the longitude and latitude coordinates to meters and
        # a tile reference

        x, y = merc.LatLonToMeters(samplepoint.y,samplepoint.x)
        tx,ty = merc.MetersToTile(x, y, 21)

        # Create a unique quadkey for each point object
                
        quadkey = merc.QuadTree(tx, ty, 21)

        # Create categorical variable for the income category

        income_category = 'HD01_VD13'         

        # Export data to the csv file

#         out_csv.loc[pandas_idx] = [geo, x, y, quadkey,income_category]
        out_csv = out_csv.append(dict([('GEO.id2',geo),('x',x),('y',y),('quadkey',quadkey),('income_category',income_category)]),ignore_index=True)
#         pandas_idx += 1
        
    for i in range(bt_100k_125k):
                
        # Choose a random longitude and latitude within the boundary box
        # and within the orginial ploygon of the census block

        while True:

            samplepoint = Point(uniform(leftmost, rightmost),uniform(bottommost, topmost))

            if samplepoint is None:
                break

            if poly.contains(samplepoint):
                break

        # Convert the longitude and latitude coordinates to meters and
        # a tile reference

        x, y = merc.LatLonToMeters(samplepoint.y,samplepoint.x)
        tx,ty = merc.MetersToTile(x, y, 21)

        # Create a unique quadkey for each point object
                
        quadkey = merc.QuadTree(tx, ty, 21)

        # Create categorical variable for the income category

        income_category = 'HD01_VD14'         

        # Export data to the csv file

#         out_csv.loc[pandas_idx] = [geo, x, y, quadkey,income_category]
        out_csv = out_csv.append(dict([('GEO.id2',geo),('x',x),('y',y),('quadkey',quadkey),('income_category',income_category)]),ignore_index=True)
#         pandas_idx += 1
        
    for i in range(bt_125k_150k):
                
        # Choose a random longitude and latitude within the boundary box
        # and within the orginial ploygon of the census block

        while True:

            samplepoint = Point(uniform(leftmost, rightmost),uniform(bottommost, topmost))

            if samplepoint is None:
                break

            if poly.contains(samplepoint):
                break

        # Convert the longitude and latitude coordinates to meters and
        # a tile reference

        x, y = merc.LatLonToMeters(samplepoint.y,samplepoint.x)
        tx,ty = merc.MetersToTile(x, y, 21)

        # Create a unique quadkey for each point object
                
        quadkey = merc.QuadTree(tx, ty, 21)

        # Create categorical variable for the income category

        income_category = 'HD01_VD15'         

        # Export data to the csv file

#         out_csv.loc[pandas_idx] = [geo, x, y, quadkey,income_category]
        out_csv = out_csv.append(dict([('GEO.id2',geo),('x',x),('y',y),('quadkey',quadkey),('income_category',income_category)]),ignore_index=True)
#         pandas_idx += 1
        
    for i in range(bt_150k_200k):
                
        # Choose a random longitude and latitude within the boundary box
        # and within the orginial ploygon of the census block

        while True:

            samplepoint = Point(uniform(leftmost, rightmost),uniform(bottommost, topmost))

            if samplepoint is None:
                break

            if poly.contains(samplepoint):
                break

        # Convert the longitude and latitude coordinates to meters and
        # a tile reference

        x, y = merc.LatLonToMeters(samplepoint.y,samplepoint.x)
        tx,ty = merc.MetersToTile(x, y, 21)

        # Create a unique quadkey for each point object
                
        quadkey = merc.QuadTree(tx, ty, 21)

        # Create categorical variable for the income category

        income_category = 'HD01_VD16'         

        # Export data to the csv file

#         out_csv.loc[pandas_idx] = [geo, x, y, quadkey,income_category]
        out_csv = out_csv.append(dict([('GEO.id2',geo),('x',x),('y',y),('quadkey',quadkey),('income_category',income_category)]),ignore_index=True)
#         pandas_idx += 1
        
    for i in range(gt_200k):
                
        # Choose a random longitude and latitude within the boundary box
        # and within the orginial ploygon of the census block

        while True:

            samplepoint = Point(uniform(leftmost, rightmost),uniform(bottommost, topmost))

            if samplepoint is None:
                break

            if poly.contains(samplepoint):
                break

        # Convert the longitude and latitude coordinates to meters and
        # a tile reference

        x, y = merc.LatLonToMeters(samplepoint.y,samplepoint.x)
        tx,ty = merc.MetersToTile(x, y, 21)

        # Create a unique quadkey for each point object
                
        quadkey = merc.QuadTree(tx, ty, 21)

        # Create categorical variable for the income category

        income_category = 'HD01_VD17'         

        # Export data to the csv file

#         out_csv.loc[pandas_idx] = [geo, x, y, quadkey,income_category]
        out_csv = out_csv.append(dict([('GEO.id2',geo),('x',x),('y',y),('quadkey',quadkey),('income_category',income_category)]),ignore_index=True)
#         pandas_idx += 1

1/450
GEO.id2            0
x                  0
y                  0
quadkey            0
income_category    0
dtype: int64


In [68]:
out_csv.to_csv("censusBlock_incomePoints.csv")